In [2]:
!pip install -U sentence-transformers
!pip install plotly==4.14.1
!pip install torch

     |████████████████████████████████| 71kB 4.1MB/s 
     |████████████████████████████████| 1.8MB 6.3MB/s 
     |████████████████████████████████| 1.2MB 17.8MB/s 
     |████████████████████████████████| 890kB 26.6MB/s 
     |████████████████████████████████| 2.9MB 35.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=db8265ed79997c148ae88d44d692d47ab3786a2ba341bd654208e19b056d6895
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c3c1269276199e68d7b7ab642b959d63348ebee04cd4f7267b2eb3c29ea83ac5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
     |████████████████████████████████| 13.2MB 248kB/s 
  Found existing installation: plotly 4.4.1
    Uninstal

In [3]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util

from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
from scipy.spatial.distance import euclidean, pdist, squareform
from sklearn import manifold          #use this for MDS computation
from google.colab import files

#visualization libs
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
% matplotlib inline

In [5]:
pd.set_option('max_colwidth', 800)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('using device: ', torch.cuda.get_device_name(device), flush=True)

model = SentenceTransformer('paraphrase-distilroberta-base-v1')

using device:  Tesla P100-PCIE-16GB


In [6]:

uploaded = files.upload()

Saving BBC_news_adjusted.csv to BBC_news_adjusted.csv
Saving jokes_stupid_wocka_relevant.csv to jokes_stupid_wocka_relevant.csv


In [180]:
dfJokes = pd.read_csv('jokes_stupid_wocka_relevant.csv', encoding='utf-8')
dfNews = pd.read_csv('BBC_news_adjusted.csv', encoding='utf-8')
dfNews.columns = ['text', 'category']


In [181]:
dfJokes['text'] = dfJokes['text'].str.lower()
dfJokes.head()


,text,category,source,score,length
0,"a brunette, a blonde, and a redhead all worked in the same office with the same female boss. every day, they noticed their boss left work early. one day, the girls decided that when the boss left, they'd leave right behind her. after all, she never called in or came back to the office when she left early, so how was she to know? the next day, they all three left the office right after the boss left. the brunette was thrilled to be home early. she did a little gardening and went to bed early. the redhead was elated to be able to get in a quick workout at the health club before meeting her dinner date. the blonde was happy, happy, happy to be home, but when she got to the bedroom she heard a muffled noise from inside. slowly, quietly, she cracked open the door and was mortified to se...",Office Jokes,stupidstuff,3.73,260
1,"bill and hillary clinton went out to dinner and when the waiter came to take their order, he asked bill how he wanted his steak, she replied, ""medium."" then the waiter said, ""how about your vegetable?"" bill replied, ""oh, she can order for herself.""",Political,stupidstuff,3.50,59
2,"the stockbroker's secretary answered his phone one morning. ""i'm sorry,"" she said, ""mr. bradford's on another line."" ""this is mr. ingram's office,"" the caller said. ""we'd like to know if he's bullish or bearish right now."" ""he's talking to his wife,"" the secretary replied. ""right now i'd say he's sheepish.""",Business,stupidstuff,2.33,85
3,"a man sent his clothing out to the local laundry. when it came back there were still stains on his underwear. the next week he enclosed a note saying, ""use more soap on underwear."" this went on for several weeks, with the underwear returning stained, and the man sending the note, ""use more soap on underwear."" finally the laundry came back with a note from the laundry man: ""use more paper on rear.""",Business,stupidstuff,3.86,91
4,"a foreman had ten very lazy men working for him. one day he decided to trick them into doing some work for a change. ""i've got a really easy job today for the laziest one among you,"" he announced. ""will the laziest man please put his hand up."" nine hands went up. ""why didn't you put your hand up?"" he asked the tenth man. ""too much trouble,"" came the reply.",Business,stupidstuff,3.00,95


In [151]:


dfJokes
dfJokes['category'] = dfJokes['category'].replace(['Political','Business', 'News / Politics', 'Sports', 'At Work', 'Office Jokes', 'Money', 'Computers', 'Tech'],['PolitcalJokes','BusinessJokes', 'PolitcalJokes', 'SportsJokes', 'WorkJokes', 'WorkJokes', 'WorkJokes' , 'TechJokes', 'TechJokes'])
dfJokes

asendatav = ['business', 'sport','politics', 'entertainment' , 'tech']
asendus = ['BusinessNews', 'SportNews', 'PoliticsNews', 'EntertainmentNews', 'TechNews' ]
dfNews['category'] = dfNews['category'].replace(asendatav,asendus)



In [165]:
dfNews.tail()

,text,category,x,y
1485,double eviction from big brother model caprice and holby city actor jeremy edwards have both left the celebrity big brother house in a surprise double eviction on friday. caprice who left in the scheduled fourth eviction having gained just 5% of the public vote afterwards said: i am so happy i am so glad i m out.,EntertainmentNews,0.621746,-0.159670
1486,dj double act revamp chart show dj duo jk and joel are taking over bbc radio 1 s flagship chart show on sunday adding showbiz news celebrity interviews and between-song banter. they hope to boost ratings for the long-running show which has been overtaken in popularity by independent radio s hit 40 uk rundown.,EntertainmentNews,0.404957,0.369954
1487,weak dollar hits reuters revenues at media group reuters slipped 11% during 2004 mainly due to the weakness of the dollar the group said. the company said it was optimistic about growth even as revenues slipped 11% from £3.24bn ($6.13bn) in 2003 to £2.89bn in 2004.,BusinessNews,0.039222,0.270206
1488,apple ipod family expands market apple has expanded its ipod family with the release of its next generation of the digital music players. its latest challenges to the growing digital music gadget market include an ipod mini model which can hold 6gb compared to a previous 4gb.,TechNews,0.133915,0.604585
1489,santy worm makes unwelcome visit thousands of website bulletin boards have been defaced by a virus that used google to spread across the net. the santy worm first appeared on 20 december and within 24 hours had successfully hit more than 40 000 websites.,TechNews,-0.152999,0.466000


In [153]:
dfJokes['category'].value_counts()

PolitcalJokes    367
WorkJokes        349
TechJokes        181
SportsJokes      166
BusinessJokes     40
Name: category, dtype: int64

In [154]:
dfJokes['category'].value_counts()

PolitcalJokes    367
WorkJokes        349
TechJokes        181
SportsJokes      166
BusinessJokes     40
Name: category, dtype: int64

In [155]:
dfNews['category'].value_counts()

SportNews            346
BusinessNews         336
PoliticsNews         274
EntertainmentNews    273
TechNews             261
Name: category, dtype: int64

In [156]:
textsNews = dfNews.text.to_list()
embsNews = model.encode(textsNews)

textsJokes = dfJokes.text.to_list()
embsJokes = model.encode(textsJokes)

In [157]:
def plt_dists(dists, df, dims=2, title=""):
  '''
  Plot distances using MDS in 2D/3D 
  dists: precomputed distance matrix
  df: dataframe
  dims: 2/3 for 2 or 3 dimensional plot, defaults to 2 for any other value passed
  title: title for the plot
  '''
  #https://community.plotly.com/t/plotly-colours-list/11730/6
  colorscale = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', 
                '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']

  #calculate MDS with number of dims passed
  mds = manifold.MDS(n_components=dims, dissimilarity="precomputed", 
                     random_state=60, max_iter=90000)
  results = mds.fit(dists)

  #get coodinates for each point
  coords = results.embedding_

  df['x'] = pd.Series(coords[:, 0])
  df['y'] = pd.Series(coords[:, 1])
  if dims == 3:
    df['z'] = pd.Series(coords[:, 2])

  label_counts = df['category'].value_counts()
  fig = go.Figure()
  idx = 0
  for cat, count in label_counts.iteritems():
    # for jokes, don't plot all categories (e.g. with less than x samples)
    #if count < 400:
    #  continue
    #df_sample = df.query('category == @cat')
    mask = df.category == cat
    if dims == 3:
      fig.add_trace(go.Scatter3d(x=df['x'][mask], y=df['y'][mask], z=df['z'][mask],
                    mode='markers', textposition="top center",
                    marker=dict(
                        size=10,
                        color=colorscale[idx],
                        opacity=0.8,
                    ), text=df['text'][mask], hoverinfo='text',
                    name=cat))
    else:
        fig.add_trace(go.Scatter(x=df['x'][mask], y=df['y'][mask],
                      mode='markers', textposition="top center",
                      marker=dict(
                          size=12,
                          color=colorscale[idx],
                          opacity=0.8,
                      ), text=df['text'][mask], hoverinfo='text',
                      name=cat))
    idx += 1      

  fig.update_layout(template="plotly_dark")
  if title!="":
    fig.update_layout(title_text=title)
  fig.show()

In [158]:
def eval_vecs(dists, df, pealkiri, category='', viz_dims=2):
  '''
  function to calculate cosine distance between each pair of input sentence vectors
  and then pass it to the visualization function

  inputs:
  dists: 
  df: dataframe containing columns 'source', 'category' and 'text'
  viz_dims: 2/3 for 2D or 3D plot
  '''

  #call the plot function on the cosine distance matrix
  plt_dists(dists, df, dims=viz_dims, 
            title=pealkiri)

In [159]:
#distances = cosine_distances(embs)
#eval_vecs(distances, dfNews, category='', viz_dims=2)

distances = cosine_distances(embsJokes)
eval_vecs(distances, dfJokes, 'Comparison of embeddings from jokes dataset', category='', viz_dims=2)

In [133]:
distances = cosine_distances(embsNews)
eval_vecs(distances, dfNews, 'Comparison of embeddings from news dataset', category='', viz_dims=2)

In [182]:
#newsTech = dfNews[ dfNews['category'] == 'BusinessNews' ]
#newsJokes = dfJokes[ dfJokes['category'] == 'BusinessJokes' ]
#vertical_stack = pd.concat([newsTech, newsJokes], axis=0)
#textTech = vertical_stack.text.to_list()
#embsTech = model.encode(textTech)
#distances = cosine_distances(embsTech)
#eval_vecs(distances, vertical_stack, 'Business', category='', viz_dims=2)


In [161]:
jokesWithoutScores = dfJokes[['text', 'category']]
koos = pd.concat([jokesWithoutScores, dfNews], ignore_index=True)
textKoos= koos.text.to_list()
embsKoos= model.encode(textKoos)



In [162]:
distances = cosine_distances(embsKoos)
eval_vecs(distances, koos, 'Comparison of total news and jokes embeddings', category='', viz_dims=2)

In [163]:
koos['category'].value_counts()

PolitcalJokes        367
WorkJokes            349
SportNews            346
BusinessNews         336
PoliticsNews         274
EntertainmentNews    273
TechNews             261
TechJokes            181
SportsJokes          166
BusinessJokes         40
Name: category, dtype: int64

In [164]:
koosIlma = koos.copy()
endised = ['PolitcalJokes',
'WorkJokes',
'SportNews',
'BusinessNews',
'PoliticsNews',
'EntertainmentNews',
'TechNews',
'TechJokes',
'SportsJokes',
'BusinessJokes']
uued = ['joke', 'joke', 'news', 'news', 'news', 'news', 'news', 'joke', 'joke', 'joke']
koosIlma['category'] = koosIlma['category'].replace(endised,uued)
textIlma= koosIlma.text.to_list()
embIlma= model.encode(textIlma)
distances = cosine_distances(embIlma)
eval_vecs(distances, koosIlma, 'Jokes vs News', category='', viz_dims=2)